In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from collections import defaultdict
from sqlalchemy import create_engine
import datetime
import random
import re


#Fake data creation

In [2]:
from numpy import integer


fake=Faker()

fake_data = defaultdict(list)

row_number=int(input("Please write the number you wish to see the row size, DON'T CRASH ME!"))



In [3]:

for i in range(row_number):
    fake_data['customer1'].append(fake.bothify(text='16######'))
    fake_data['amount'].append(fake.pyfloat(right_digits=2,positive=True,min_value=2.00,max_value=10000.00))
    fake_data['timestamp'].append(fake.date_time_ad(start_datetime=datetime.datetime(2022,9,14)))
    

In [4]:
copy_list=fake_data['customer1'].copy()
random.shuffle(copy_list)
fake_data['customer2']=copy_list
fake_data

defaultdict(list,
            {'customer1': ['16210714',
              '16272462',
              '16814287',
              '16419300',
              '16633040',
              '16083753',
              '16036182',
              '16827025',
              '16067162',
              '16250990'],
             'amount': [9681.8,
              6041.2,
              892.2,
              2161.1,
              2507.54,
              1728.86,
              4963.82,
              9915.0,
              8129.99,
              4649.6],
             'timestamp': [datetime.datetime(2022, 9, 15, 21, 21, 51),
              datetime.datetime(2022, 10, 1, 20, 26, 48),
              datetime.datetime(2022, 9, 26, 11, 56, 30),
              datetime.datetime(2022, 10, 28, 10, 24, 48),
              datetime.datetime(2022, 10, 18, 17, 26, 24),
              datetime.datetime(2022, 10, 7, 14, 16, 45),
              datetime.datetime(2022, 10, 2, 3, 11, 14),
              datetime.datetime(2022, 10, 30, 6, 46, 3

#Adding dict to dataframe

In [30]:
df=pd.DataFrame(fake_data)

df=df[['customer1','customer2','amount','timestamp']]

print(df)

  customer1 customer2   amount           timestamp
0  16210714  16083753  9681.80 2022-09-15 21:21:51
1  16272462  16419300  6041.20 2022-10-01 20:26:48
2  16814287  16814287   892.20 2022-09-26 11:56:30
3  16419300  16067162  2161.10 2022-10-28 10:24:48
4  16633040  16633040  2507.54 2022-10-18 17:26:24
5  16083753  16250990  1728.86 2022-10-07 14:16:45
6  16036182  16827025  4963.82 2022-10-02 03:11:14
7  16827025  16210714  9915.00 2022-10-30 06:46:31
8  16067162  16272462  8129.99 2022-10-18 07:16:56
9  16250990  16036182  4649.60 2022-09-14 17:00:46


In [35]:
for i in range (len(df['customer1'])):
        a=df['customer1'][i]
        b=df['customer2'][i]
        rnd =random.choices(df['customer1'])
        print(type(rnd[i]))
        """
        if a==b:
            if rnd == df['customer1'][i]:
                rnd =random.choices(df['customer1'])
                print(type(rnd))
                df['customer2'][i]=str(rnd)
            df['customer2'][i]=str(rnd)
            """
    

        
            
       


    





IndexError: list index out of range

In [28]:
df['customer2']

0      16083753
1      16419300
2    [16210714]
3      16067162
4    [16083753]
5      16250990
6      16827025
7      16210714
8      16272462
9      16036182
Name: customer2, dtype: object